In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from xgboost import XGBClassifier
import joblib
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import random
import requests
import json
from time import sleep


# Reading dataset

In [7]:
loans_df = pd.read_csv('LoansTrainingSetV2.csv')
loans_df.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,6cf51492-02a2-423e-b93d-676f05b9ad53,7c202b37-2add-44e8-9aea-d5b119aea935,Loan Refused,12232,Short Term,7280.0,< 1 year,Rent,46643.0,Debt Consolidation,777.39,18.0,10.0,12,0,6762,7946,0.0,0.0
1,552e7ade-4292-4354-9ff9-c48031697d72,e7217b0a-07ac-47dd-b379-577b5a35b7c6,Loan Refused,25014,Long Term,7330.0,10+ years,Home Mortgage,81099.0,Debt Consolidation,892.09,26.7,NaN,14,0,35706,77961,0.0,0.0
2,9b5e32b3-8d76-4801-afc8-d729d5a2e6b9,0a62fc41-16c8-40b5-92ff-9e4b763ce714,Loan Refused,16117,Short Term,7240.0,9 years,Home Mortgage,60438.0,Home Improvements,1244.02,16.7,32.0,11,1,11275,14815,1.0,0.0
3,5419b7c7-ac11-4be2-a8a7-b131fb6d6dbe,30f36c59-5182-4482-8bbb-5b736849ae43,Loan Refused,11716,Short Term,7400.0,3 years,Rent,34171.0,Debt Consolidation,990.94,10.0,NaN,21,0,7009,43533,0.0,0.0
4,1450910f-9495-4fc9-afaf-9bdf4b9821df,70c26012-bba5-42c0-8dcb-75295ada31bb,Loan Refused,9789,Long Term,6860.0,10+ years,Home Mortgage,47003.0,Home Improvements,503.71,16.7,25.0,13,1,16913,19553,1.0,0.0


# Preprocessing Data

In [8]:
loans_df = loans_df.drop(columns=['Loan ID',  'Customer ID', 'Loan Status'])
ftr = loans_df['Maximum Open Credit'] ==  '#VALUE!'
loans_df.loc[ftr, 'Maximum Open Credit'] = np.nan
loans_df['Maximum Open Credit'] = loans_df['Maximum Open Credit'].astype('float64')
loans_df['Monthly Debt'] = loans_df['Monthly Debt'].str.replace('$', '')
loans_df['Monthly Debt'] = loans_df['Monthly Debt'].astype('float64')

cat_cols = loans_df.columns[loans_df.dtypes == 'object']
cat_encoder = OrdinalEncoder()
loans_df[cat_cols] = cat_encoder.fit_transform(loans_df[cat_cols])
loans_df.dtypes

Current Loan Amount               int64
Term                            float64
Credit Score                    float64
Years in current job            float64
Home Ownership                  float64
Annual Income                   float64
Purpose                         float64
Monthly Debt                    float64
Years of Credit History         float64
Months since last delinquent    float64
Number of Open Accounts           int64
Number of Credit Problems         int64
Current Credit Balance            int64
Maximum Open Credit             float64
Bankruptcies                    float64
Tax Liens                       float64
dtype: object

In [9]:
loans_df = loans_df.dropna()

In [10]:
# making sure the prediction dataset follows the same structure
inference_columns = joblib.load('inference_columns.joblib')

In [11]:
loans_df = loans_df[inference_columns]

### Testing predictions

In [94]:
endpoint = 'http://127.0.0.1:8080/eligibility'

while True:
    # sample a record from the test set
    r_idx = random.randrange(0, loans_df.shape[0] - 1)
    record = json.dumps(loans_df.iloc[r_idx,:].tolist())
    json_record = {'data': record}
    
    # Query the endpoint
    response = requests.post(endpoint, json=json_record)
    if response.status_code == 200:
        print(response.text)
    sleep(1)

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan approved"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan approved"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Suc

KeyboardInterrupt: 

# Testting deployed endpoint on GCP

In [50]:
token = !gcloud auth print-identity-token
headers = {"Authorization": f"Bearer {token[0]}"}

In [53]:
url = 'https://loan-forecaster-5on7t3ikfa-rj.a.run.app/eligibility'

while True:
    # sample a record from the test set
    r_idx = random.randrange(0, loans_df.shape[0] - 1)
    record = json.dumps(loans_df.iloc[r_idx,:].tolist())
    data = {'data': record}
    
    # Query the endpoint
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        print(response.text)
    sleep(1)

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan approved"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan approved"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan approved"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan Denied"}

{"code":"200","msg":"Predicted Successfully","prediction":"Loan approved"}

{"code":"200","msg":"Predicted

KeyboardInterrupt: 